## 0. Import MNE & raw data

In [2]:
import mne
import pandas as pd
import os
mne.set_log_level('error')  # reduce extraneous MNE output

# 1. Filtering Raw Data

## a. Define directories

In [4]:
file_name = 'eeg1'
eeg_file = file_name + '.EDF'

# Data directory
data_dir = '/Users/khanhha/Desktop/EEG_data/data'
raw_dir = '/Users/khanhha/Desktop/EEG_data/raw'
filt_dir = '/Users/khanhha/Desktop/EEG_data/raw_filt'


## b. Filter & Save

In [4]:
# Load in the raw data
file_path = os.path.join(data_dir, eeg_file)
raw = mne.io.read_raw_edf(file_path, preload=True)

# # Set electrode locations
# montage = 'easycap-M1' # Electrode position file
# raw.set_montage(montage)

# Filter settings
low_cut = 1
hi_cut  = 30

# Filtering data
raw_filt = raw.copy().filter(low_cut, hi_cut)

# Saving the filtered and unfiltered as .fif files so we dont need to open the edf again
# Define file paths
raw_path = raw_dir + '/' + file_name + '-raw.fif'
filt_path = filt_dir + '/' + file_name + '-filt-raw.fif'
# Save the files as fif
raw.save(raw_path, overwrite=True)
raw_filt.save(filt_path, overwrite=True)


# 2. Artifacts Removal with Independent Components Analysis

In [8]:
raw = mne.io.read_raw_fif(raw_dir + '/' + file_name + '-raw.fif', preload=True)

In [5]:
raw_ica = mne.io.read_raw_fif(filt_dir + '/' + file_name + '-filt-raw.fif', preload=True)

Set Montage

In [14]:
info = raw_ica.info
info["ch_names"]

['C3',
 'C4',
 'O1',
 'O2',
 'A1',
 'A2',
 'Cz',
 'F3',
 'F4',
 'F7',
 'F8',
 'Fz',
 'Fp1',
 'Fp2',
 'Fpz',
 'P3',
 'P4',
 'Pz',
 'T3',
 'T4',
 'T5',
 'T6',
 'ECGL',
 'ECGR']

In [32]:
print(raw_ica.info['chs'])
print(raw_ica.info['dig'])
# montage = mne.channels.make_standard_montage('standard_1005')
# raw_ica.set_montage(montage)


[{'scanno': 1, 'logno': 1, 'kind': 2 (FIFFV_EEG_CH), 'range': 1.0, 'cal': 1.0, 'coil_type': 1 (FIFFV_COIL_EEG), 'loc': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]), 'unit': 107 (FIFF_UNIT_V), 'unit_mul': 0 (FIFF_UNITM_NONE), 'ch_name': 'C3', 'coord_frame': 4 (FIFFV_COORD_HEAD)}, {'scanno': 2, 'logno': 2, 'kind': 2 (FIFFV_EEG_CH), 'range': 1.0, 'cal': 1.0, 'coil_type': 1 (FIFFV_COIL_EEG), 'loc': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]), 'unit': 107 (FIFF_UNIT_V), 'unit_mul': 0 (FIFF_UNITM_NONE), 'ch_name': 'C4', 'coord_frame': 4 (FIFFV_COORD_HEAD)}, {'scanno': 3, 'logno': 3, 'kind': 2 (FIFFV_EEG_CH), 'range': 1.0, 'cal': 1.0, 'coil_type': 1 (FIFFV_COIL_EEG), 'loc': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]), 'unit': 107 (FIFF_UNIT_V), 'unit_mul': 0 (FIFF_UNITM_NONE), 'ch_name': 'O1', 'coord_frame': 4 (FIFFV_COORD_HEAD)}, {'scanno': 4, 'logno': 4, 'kind': 2 (FIFFV_EEG_CH), 'range': 1.0, 'cal': 1.0, 'coil_type': 1 

In [12]:
montage = "standard_1020"
raw_ica.set_montage(montage)

### Segment data for ICA
Break the data into epochs of 1s  
Epochs - segments of data that timed-locked to experimental events of interest

In [15]:
tstep = 1.0 #seconds
events_ica = mne.make_fixed_length_events(raw_ica, duration=tstep)
epochs_ica = mne.Epochs(raw_ica, events_ica,
                        tmin=0.0, tmax=tstep,
                        baseline=None,
                        preload=True)

In [16]:
epochs_ica

Number of events,69709
Events,1: 69709
Time range,0.000 – 1.000 s
Baseline,off


In [18]:
from autoreject import AutoReject

ar = AutoReject(n_interpolate=[1, 2, 4],
                random_state=42,
                picks=mne.pick_types(epochs_ica.info, 
                                     eeg=True,
                                     eog=False
                                    ),
                n_jobs=-1, 
                verbose=False
                )

ar.fit(epochs_ica)

reject_log = ar.get_reject_log(epochs_ica)

RuntimeError: Valid channel positions are needed for autoreject to work

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=[15, 5])
reject_log.plot('horizontal', ax=ax, aspect='auto')
plt.show()

## Others

### Plots

In [1]:
# raw.compute_psd().plot()

### Data Preprocessing Information from Johnsen et al.
*Common patterns of EEG reactivity in post-anoxic coma identified by quantitative analyses - ScienceDirect* 

EEG Recordings:  
Recorded 20-30 mins during hypothermia from 12-24 hours after target temp reached.  
Common average reference montage with 19 electrodes  
Sampling rate: 256 Hz  
Band-pass filter: 0.5-70 Hz  
“Nicolet One” software from “Natus” ver 5.82

EEG analyses  
2-s epochs x 14 with 50% overlap  
From the 1st full second after the stimulation start to 15s after stimulation start.  
Reference epoch: 2 seconds before the stimulation start  
For example:  
Stimulation time: 12:26:15,45  
Reference epoch: 12:26:13,00 - 12:26:15,00  
Epoch 1: 12:26:16,00 - 12:26:18,00  
Epoch 2: 12:26:17,00 - 12:26:19,00  
…  
Epoch 14: 12:26:29,00 - 12:26:31,00  

In [26]:
# Filter settings
low_cut = 1
hi_cut  = 30

raw_filt = raw.copy().filter(low_cut, hi_cut)

In [22]:
from eeg_positions import get_alias_mapping, get_available_elec_names, get_elec_coords, plot_coords

In [24]:
elec_names = get_available_elec_names()
print(elec_names[::-1])

['M2', 'M1', 'A2', 'A1', 'NAS', 'RPA', 'LPA', 'POO8h', 'POO6', 'POO6h', 'POO4', 'POO4h', 'POO2', 'POO2h', 'POO1h', 'POO1', 'POO3h', 'POO3', 'POO5h', 'POO5', 'POO7h', 'PO8h', 'PO6', 'PO6h', 'PO4', 'PO4h', 'PO2', 'PO2h', 'PO1h', 'PO1', 'PO3h', 'PO3', 'PO5h', 'PO5', 'PO7h', 'PPO8h', 'PPO6', 'PPO6h', 'PPO4', 'PPO4h', 'PPO2', 'PPO2h', 'PPO1h', 'PPO1', 'PPO3h', 'PPO3', 'PPO5h', 'PPO5', 'PPO7h', 'P8h', 'P6', 'P6h', 'P4', 'P4h', 'P2', 'P2h', 'P1h', 'P1', 'P3h', 'P3', 'P5h', 'P5', 'P7h', 'TPP8h', 'CPP6', 'CPP6h', 'CPP4', 'CPP4h', 'CPP2', 'CPP2h', 'CPP1h', 'CPP1', 'CPP3h', 'CPP3', 'CPP5h', 'CPP5', 'TPP7h', 'TP8h', 'CP6', 'CP6h', 'CP4', 'CP4h', 'CP2', 'CP2h', 'CP1h', 'CP1', 'CP3h', 'CP3', 'CP5h', 'CP5', 'TP7h', 'TTP8h', 'CCP6', 'CCP6h', 'CCP4', 'CCP4h', 'CCP2', 'CCP2h', 'CCP1h', 'CCP1', 'CCP3h', 'CCP3', 'CCP5h', 'CCP5', 'TTP7h', 'FTT8h', 'FCC6', 'FCC6h', 'FCC4', 'FCC4h', 'FCC2', 'FCC2h', 'FCC1h', 'FCC1', 'FCC3h', 'FCC3', 'FCC5h', 'FCC5', 'FTT7h', 'FT8h', 'FC6', 'FC6h', 'FC4', 'FC4h', 'FC2', 'FC2h

In [25]:
alias_mapping = get_alias_mapping()
for key, val in alias_mapping.items():
    print(f"{key}: {val}")

A1: LPA+(-0.1, -0.01, -0.01)
A2: RPA+(0.1, -0.01, -0.01)
M1: TP9
M2: TP10


In [30]:
# Just use `elec_names`
coords = get_elec_coords(
    elec_names=['C3', 'C4', 'O1', 'O2', 'A1', 'A2', 'Cz', 'F3', 'F4', 'F7', 'F8', 'Fz', 
            'Fp1', 'Fp2', 'Fpz', 'P3', 'P4', 'Pz'],
            # 'T3', 'ECGL', 'ECGR', 'T6', 'T5', 'T4'
    drop_landmarks=False,
    dim="3d",
)

/Users/khanhha/anaconda3/envs/cheme/lib/python3.9/site-packages/eeg_positions/utils.py:242: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  x = float(df[df["label"] == label].x)
/Users/khanhha/anaconda3/envs/cheme/lib/python3.9/site-packages/eeg_positions/utils.py:243: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  y = float(df[df["label"] == label].y)
/Users/khanhha/anaconda3/envs/cheme/lib/python3.9/site-packages/eeg_positions/utils.py:244: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  z = float(df[df["label"] == label].z)
/Users/khanhha/anaconda3/envs/cheme/lib/python3.9/site-packages/eeg_positions/utils.py:242: FutureWarning: Calling float on a single element Series is deprecated and will rais

In [31]:
coords

,label,x,y,z
0,C3,-0.5878,0.0000,0.809
1,C4,0.5878,0.0000,0.809
2,O1,-0.2939,-0.9045,0.309
3,O2,0.2939,-0.9045,0.309
4,A1,NaN,NaN,NaN
5,A2,NaN,NaN,NaN
6,Cz,0.0000,0.0000,1.000
7,F3,-0.4591,0.5800,0.673
8,F4,0.4591,0.5800,0.673
9,F7,-0.7695,0.5590,0.309
